# **GRANT WRITING ASSISTANT**


##NIH DATA RETRIEVAL


In [1]:
import requests
from datetime import datetime, timedelta

def fetch_nih_funding_data(keyword, limit):
    url = "https://api.reporter.nih.gov/v2/projects/search"

    headers = {"Content-Type": "application/json"}
    params = {
        "criteria": {
        "advanced_text_search": {
            "operator": "and",
            "search_field": "projecttitle,projectterms,projectabstracts",
            "search_text": keyword
            } },
        "limit" : limit
      }

    try:
        response = requests.post(url, headers=headers, json=params)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"Error fetching NIH data: {e}")
        return None


##NIH DATA TO DRIVE

In [2]:
from google.colab import drive
import pandas as pd

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Prompt the user for a keyword to search
keyword = input("Enter the keyword for NIH funding search: ")
limit = input("Enter the limit for NIH funding searches: ")

# Convert the limit to an integer
try:
    limit = int(limit)
except ValueError:
    print("Invalid limit value. Please enter a numeric value.")
    limit = 10  # Default limit

# Example usage: Fetch NIH funding data
nih_data_debug = fetch_nih_funding_data(keyword, limit)

# Collecting project data
project_data = []

if nih_data_debug and 'results' in nih_data_debug:
    for project in nih_data_debug['results']:
        # Collect each project's data in a dictionary
        project_info = {
            "Project Title": project.get("project_title", "No title provided"),
            "Abstract": project.get("abstract_text", "No abstract provided"),
            "Organization": project.get("organization", {}).get("org_name", "No organization"),
            "Award Amount": project.get("award_amount", "Not specified"),
            "Project Start Date": project.get("project_start_date", "Not specified")

        }
        # Append to list
        project_data.append(project_info)

    # Step 2: Create DataFrame and save to Google Drive
    file_name = f"/content/drive/MyDrive/nih_funding_data_{keyword}.csv"
    df = pd.DataFrame(project_data)

    # Save to CSV
    df.to_csv(file_name, index=False)
    print(f"Data saved to Google Drive at {file_name}")
else:
    print("No funding data returned or invalid response structure.")


Mounted at /content/drive
Enter the keyword for NIH funding search: education
Enter the limit for NIH funding searches: 20
Data saved to Google Drive at /content/drive/MyDrive/nih_funding_data_education.csv


In [3]:
from google.colab import drive
import pandas as pd
import requests
from datetime import datetime, timedelta

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Function to fetch NSF funding data
def fetch_nsf_funding_data(keyword, limit=10):
    url = "https://api.nsf.gov/services/v1/awards.json"

    # Calculate the date range for the past year
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)

    # Format the dates (mm/dd/yyyy)
    start_date_str = start_date.strftime('%m/%d/%Y')
    end_date_str = end_date.strftime('%m/%d/%Y')

    params = {
        "keyword": keyword,

        "dateStart": start_date_str,
        "dateEnd": end_date_str,
        "rpp": limit,
        "offset": 1,
        "printFields": "id,awardeeName,title,date,fundsObligatedAmt,piFirstName,piLastName"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching NSF data: {e}")
        return None

# Prompt the user for a keyword and limit
keyword = input("Enter the keyword for NSF funding search: ")
limit = input("Enter the limit for NSF funding searches: ")

# Convert the limit to an integer
try:
    limit = int(limit)
except ValueError:
    print("Invalid limit value. Defaulting to 10.")
    limit = 10

# Fetch NSF funding data
nsf_data = fetch_nsf_funding_data(keyword, limit)

# Step 2: Collect project data and save to Google Drive
project_data = []

if nsf_data and "response" in nsf_data:
    for award in nsf_data["response"].get("award", []):
        # Collect each project's data in a dictionary
        project_info = {
            "Project Title": project.get("title", "No title provided"),
            "Abstract": project.get("abstractText", "No abstract provided"),
            "Organization": project.get("awardeeName", {}).get("org_name", "No organization"),
            "Award Amount": project.get("fundsObligatedAmt", "Not specified"),
            "Project Start Date": project.get("startDate", "Not specified"),
        }
        project_data.append(project_info)

    # Create a DataFrame and save it to a CSV file in Google Drive
    file_name = f"/content/drive/MyDrive/nsf_funding_data_{keyword}.csv"
    df = pd.DataFrame(project_data)

    # Save the DataFrame as a CSV file
    df.to_csv(file_name, index=False)
    print(f"Data saved to Google Drive at {file_name}")
else:
    print("No funding data returned or invalid response structure.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter the keyword for NSF funding search: education
Enter the limit for NSF funding searches: 20
Data saved to Google Drive at /content/drive/MyDrive/nsf_funding_data_education.csv
